# Property Scrape

In [107]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [108]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [38]:
public_df = pd.read_csv('../data/raw/wcmsp-185933.csv')
public_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 16 columns):
Organization Name                           158 non-null object
Property Name                               158 non-null object
Address                                     158 non-null object
Postal Code                                 158 non-null int64
ENERGY STAR Score                           158 non-null int64
Primary Property Type                       158 non-null object
Floor Area (Buildings) (ft²)                158 non-null int64
Floor Area (Parking) (ft²)                  158 non-null int64
Year Built                                  158 non-null int64
Total GHG Emissions (Metric Tons CO2e)      158 non-null int64
Site EUI (kBtu/ft²)                         158 non-null int64
Weather Normalized Site EUI (kBtu/ft²)      158 non-null int64
Source EUI (kBtu/ft²)                       158 non-null int64
Weather Normalized Source EUI (kBtu/ft²)    158 non-null int64
Water Use

In [104]:
private_df = pd.read_csv('../data/raw/wcmsp-185935.csv')

In [111]:
private_df.head()

,Address,Postal Code,ENERGY STAR Score,Primary Property Type,Floor Area (Building) (ft²),Floor Area (Parking) (ft²),Year Built,Total GHG Emissions (Metric Tons CO2e),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Water Use (kgal),Unnamed: 13,search_address
0,201 Chicago Ave S,55415,36,Office,62400,0,2012,971,115,118,251,252,708,NaN,201 Chicago ave S
1,1011 Nicollet Mall,55403,68,Office,59479,0,1920,986,90,93,256,266,882,NaN,1011 Nicollet Mall
2,4701 Beard Ave S,55410,0,Other,59172,0,1914,1622,157,160,427,428,163,NaN,4701 Beard ave S
3,8 60Th St W,55419,76,Worship Facility,60640,62000,1959,268,50,54,77,81,674,NaN,8 60Th St W
4,700 10Th Ave S,55415,89,Office,60528,0,1920,504,74,77,136,140,414,NaN,700 10Th ave S


In [105]:
# Clean the address strings to the required format for Hennepin County Property Info search

def clean_addresses(df):
    df['search_address'].replace('Avenue', 'ave', inplace=True, regex=True)
    df['search_address'].replace('Ave\.', 'ave', inplace=True, regex=True)
    df['search_address'].replace('Ave', 'ave', inplace=True, regex=True)
    df['search_address'].replace('NE', 'n e', inplace=True, regex=True)
    df['search_address'].replace('SE', 's e', inplace=True, regex=True)
    df['search_address'].replace('Southeast', 's e', inplace=True, regex=True)
    df['search_address'].replace('Northeast', 'n e', inplace=True, regex=True)
    df['search_address'].replace('Southwest', 's w', inplace=True, regex=True)
    df['search_address'].replace('Curve', 'cur', inplace=True, regex=True)
    df['search_address'].replace('Parkway', 'pkwy', inplace=True, regex=True)
    df['search_address'].replace('Terrace', 'ter', inplace=True, regex=True)
    df['search_address'].replace('East', 'e', inplace=True, regex=True)
    df['search_address'].replace('S\.', 's', inplace=True, regex=True)
    df['search_address'].replace('South', 's', inplace=True, regex=True)
    df['search_address'].replace('Boulevard', 'blvd', inplace=True, regex=True)
    df['search_address'].replace('Curve', 'cur', inplace=True, regex=True)
    df['search_address'].replace('Drive', 'dr', inplace=True, regex=True)
    df['search_address'].replace('Trail', 'tr', inplace=True, regex=True)
    df['search_address'].replace('North', 'n', inplace=True, regex=True)
    df['search_address'].replace('Circle', 'cir', inplace=True, regex=True)
    df['search_address'].replace('Highway', 'hwy', inplace=True, regex=True)
    df['search_address'].replace('Road', 'rd', inplace=True, regex=True)
    df['search_address'].replace('Court', 'ct', inplace=True, regex=True)
    df['search_address'].replace('Lane', 'ln', inplace=True, regex=True)
    df['search_address'].replace('St\.', 'st', inplace=True, regex=True)
    df['search_address'].replace('Street', 'st', inplace=True, regex=True)
    df['search_address'].replace('street', 'st', inplace=True, regex=True)
    df['search_address'].replace('steet', 'st', inplace=True, regex=True)
    df['search_address'].replace('Northwest', 'n w', inplace=True, regex=True)
    df['search_address'].replace('West', 'w', inplace=True, regex=True)
    df['search_address'].replace('County', 'co', inplace=True, regex=True)
    df['search_address'].replace('Number', 'no', inplace=True, regex=True)
    df['search_address'].replace('Sixth', '6th', inplace=True, regex=True)

In [112]:
private_df['search_address'] = private_df['Address']
clean_addresses(private_df)
private_df.search_address

0                           201 Chicago ave S
1                          1011 Nicollet Mall
2                            4701 Beard ave S
3                                 8 60Th St W
4                              700 10Th ave S
5                               805 2Nd ave S
6                                123 3Rd St N
7                          700 Humboldt ave N
8                                700 3Rd St S
9                               525 w 54th St
10                              920 Lake St E
11                              560 6Th ave N
12                          3800 Pleasant ave
13                               84 10th St S
14                             1455 Lake St W
15                               233 Park ave
16                            2850 26Th ave S
17                        300 Industrial Blvd
18                              910 26Th St E
19                           2800 Chicago ave
20                          2410 Irving ave N
21                          4021 T

In [80]:
public_df['search_address'] = public_df['Address']
clean_addresses(public_df)
public_df.search_address

0               1800 Chicago s
1             300 Nicolet Mall
2            330 south 12th st
3                 110 s 4th st
4            530 Chicago ave s
5                    300 s 6th
6           525 Portland ave s
7                 701 Park ave
8                    626 s 6th
9              1313 Penn ave N
10            1315 Lowry ave n
11                 401 4 ave s
12         2810 Nicollet ave s
13            935 Hennepin ave
14             915 LaSalle ave
15                350 s 5th st
16               309 2nd ave s
17           1200 Currie ave n
18             25 37th ave n e
19               333 3rd ave s
20               121 E 15TH ST
21            415 south 3rd st
22                424 4th st s
23           4131 Dupont ave n
24                25 s 11th st
25          6036 Harriet ave s
26                 31 n 9th st
27              1901 e 26th st
28              1030 2nd ave s
29              1001 2nd ave s
                ...           
128             3017 E 31st st
129     

In [113]:
import re
import string

def selenium_search_url(driver, search_url, address):
    driver.get(search_url)

    address_num = address.split(" ")[0]
    address_street = address.split(" ")[1:]
    address_box = driver.find_element_by_id("INPUT1")
    address_box.send_keys(address_num)
    street_box = driver.find_element_by_id("INPUT3")
    street_box.send_keys(address_street)

    submit_button = driver.find_element_by_id("INPUT4")
    submit_button.click()
    
    property_url = driver.current_url
    print(property_url)

    content = (driver.page_source).encode('ascii', 'ignore')
    return content

def create_soup(content):
    soup = BeautifulSoup(content,"lxml")
    return soup

def search_multiple_properties(address, trs, data):
    for tr in trs:
        match = re.search(r'(pid\=)(\d*)(\"\>)', str(tr))
        if match:
            print(match.group(2))
            url = "http://www16.co.hennepin.mn.us/pins/pidresult.jsp?pid=" + match.group(2)
            driver.get(url)
            content = driver.page_source
            soup = create_soup(content)
            parse_property_page(address, soup, data) 
    return data

def parse_property_page(address, soup, data):
    new_record = {}
    
    if "Direct Debit program" in soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[0].text.strip():
        property_id = int(soup.find(id='TABLE_DETAILS').find_all('tr')[7].find_all('td')[1].text.strip().replace('-',''))
        new_record['address'] = address 
        new_record['property_id'] = property_id 
        new_record['construction_year'] = soup.find(id='TABLE_DETAILS').find_all('tr')[10].find_all('td')[4].text.strip() 
        new_record['owner_name'] = soup.find(id='TABLE_DETAILS').find_all('tr')[15].find_all('td')[1].text.strip()

        if "NO SALE INFORMATION ON FILE" in soup.find(id='TABLE_DETAILS').find_all('tr')[24].find_all('td')[0].text.strip():
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[39].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[40].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[42].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[45].find_all('td')[1].text.strip()
        else:
            new_record['sale_date'] = soup.find(id='TABLE_DETAILS').find_all('tr')[24].find_all('td')[1].text.strip()
            new_record['sale_price'] = soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[1].text.strip()
            new_record['transaction_type'] = soup.find(id='TABLE_DETAILS').find_all('tr')[26].find_all('td')[1].text.strip()
            #positions with NO sales section
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[42].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[44].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[47].find_all('td')[1].text.strip()
    
    elif "State Copy to be used when filing for the" in soup.find(id='TABLE_DETAILS').find_all('tr')[4].find_all('td')[0].text.strip():
        property_id = int(soup.find(id='TABLE_DETAILS').find_all('tr')[8].find_all('td')[1].text.strip().replace('-',''))
        new_record['address'] = address 
        new_record['property_id'] = property_id 
        new_record['construction_year'] = soup.find(id='TABLE_DETAILS').find_all('tr')[11].find_all('td')[4].text.strip() 
        new_record['owner_name'] = soup.find(id='TABLE_DETAILS').find_all('tr')[16].find_all('td')[1].text.strip()
        
        if "NO SALE INFORMATION ON FILE" in soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[0].text.strip():
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[40].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[43].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[46].find_all('td')[1].text.strip()
        else:
            new_record['sale_date'] = soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[1].text.strip()
            new_record['sale_price'] = soup.find(id='TABLE_DETAILS').find_all('tr')[26].find_all('td')[1].text.strip()
            new_record['transaction_type'] = soup.find(id='TABLE_DETAILS').find_all('tr')[27].find_all('td')[1].text.strip()
            #positions with sales section
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[42].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[43].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[45].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[48].find_all('td')[1].text.strip()
    else:  
        property_id = int(soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text.strip().replace('-',''))
        new_record['address'] = address 
        new_record['property_id'] = property_id 
        new_record['construction_year'] = soup.find(id='TABLE_DETAILS').find_all('tr')[9].find_all('td')[4].text.strip() 
        new_record['owner_name'] = soup.find(id='TABLE_DETAILS').find_all('tr')[14].find_all('td')[1].text.strip()
        
        if "NO SALE INFORMATION ON FILE" in soup.find(id='TABLE_DETAILS').find_all('tr')[23].find_all('td')[0].text.strip():
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[38].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[39].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[44].find_all('td')[1].text.strip()
        else:
            new_record['sale_date'] = soup.find(id='TABLE_DETAILS').find_all('tr')[23].find_all('td')[1].text.strip()
            new_record['sale_price'] = soup.find(id='TABLE_DETAILS').find_all('tr')[24].find_all('td')[1].text.strip()
            new_record['transaction_type'] = soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[1].text.strip()
            #positions with NO sales section
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[40].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[43].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[46].find_all('td')[1].text.strip()          
    
    new_record['land_market_value'] = soup.find(id='Table_Detail').find_all('tr')[3].find_all('td')[1].text.strip() 
    new_record['building_market_value'] = soup.find(id='Table_Detail').find_all('tr')[4].find_all('td')[1].text.strip() 
    new_record['total_market_value'] = soup.find(id='Table_Detail').find_all('tr')[6].find_all('td')[1].text.strip()
    new_record['homestead_mrkt_val_excl'] = soup.find(id='Table_Detail').find_all('tr')[9].find_all('td')[1].text.strip()
    new_record['property_type'] = soup.find(id='Table_Detail').find_all('tr')[11].find_all('td')[1].text.strip()
    new_record['homestead_status'] = soup.find(id='Table_Detail').find_all('tr')[12].find_all('td')[1].text.strip()

    data[property_id] = new_record
    return data

In [114]:
def run_scraper(df):
    chromedriver = "./chromedriver"
    driver = webdriver.Chrome(chromedriver)
    search_url="http://www16.co.hennepin.mn.us/pins/addrsrch.jsp"

    data = {}
    for index, row in df.iterrows():
        try:
            content = selenium_search_url(driver, search_url, row['search_address'])
            soup = create_soup(content)

            if soup.find(id='TABLE_DETAILS') and soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text:
                print("Found " + row['Address'] + ": " + soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text)
                parse_property_page(row['Address'], soup, data)
            elif soup.find(class_="notfound", string="No records found"):
                print("No records found for " +  row['Address'])
            elif soup.find("span", string="Click on property ID for details"):
                print(row['Address'] + " has multiple properties")
                select = driver.find_element_by_name("ps")
                select.send_keys('100')
                submit_button = driver.find_element_by_id("INPUT4")
                submit_button.click()

                content = (driver.page_source).encode('ascii', 'ignore')
                soup = create_soup(content)
                trs = soup.find(id='TABLE1').find_all('tr')
                search_multiple_properties(row['Address'], trs, data)
            else:
                print("Something went wrong " + row['Address'])
        except Exception:
            print("Exception on " +  row['Address'])
            pass
    return data
    

In [90]:
public_property_info = run_scraper(public_df)

http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 1800 Chicago S.: 26-029-24-33-0002
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 300 Nicolet Mall: 22-029-24-44-0012
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 330 south 12th st
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 110 S. 4th St.
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 530 Chicago Avenue South: 26-029-24-21-0070
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 300 S. Sixth
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 525 Portland Ave South
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
701 Park Avenue has multiple properties
2602924240052
Exception on 701 Park Avenue
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 626 S. Sixth
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 1313 Penn Ave N: 17-029-24-44-0212
http://www16.co.hennepin.mn.us/pins/addrresult

In [91]:
import pickle

with open('0201_public_prop_dict.pkl', 'wb') as picklefile:
    pickle.dump(public_property_info, picklefile)

In [115]:
private_property_info = run_scraper(private_df)


http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 201 Chicago Ave S: 23-029-24-43-0394
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 1011 Nicollet Mall
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 4701 Beard Ave S: 17-028-24-21-0100
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
Found 8 60Th St W: 22-028-24-34-0118
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
700 10Th Ave S has multiple properties
2602924240086
2602924240086
2402924240052
2402924240176
2402924240185
2402924240186
2402924240187
2402924240188
2402924240189
2402924240190
2402924240191
2402924240192
2402924240193
2402924240194
2402924240177
2402924240195
2402924240196
2402924240197
2402924240198
2402924240199
2402924240178
2402924240179
2402924240180
2402924240181
2402924240182
2402924240183
2402924240184
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
805 2Nd Ave S has multiple properties
2702924110085
2702924110085
2411722130087
http://www16.co.hennepin.mn.us/pins

In [116]:
#41 of 258 private properties were not found
import pickle

with open('0201_private_prop_dict.pkl', 'wb') as picklefile:
    pickle.dump(private_property_info, picklefile)

In [36]:
clean_addresses(public_df)
public_df.sample(5)

,Organization Name,Property Name,Address,Postal Code,ENERGY STAR Score,Primary Property Type,Floor Area (Buildings) (ft²),Floor Area (Parking) (ft²),Year Built,Total GHG Emissions (Metric Tons CO2e),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Water Use (kgal),Unnamed: 15,search_address
77,Minneapolis Park and Recreation Board,Northeast Park - Demolished [Mn012187],1616 Buchanon St NE,55413,0,Other - Entertainment/Public Assembly,0,0,1970,0,0,0,0,0,0,NaN,1616 Buchanon St NE
134,Minneapolis Public Schools,North Senior High,1500 James Ave N,55411,8,K-12 School,290000,0,1972,3888,125,131,224,231,5118,NaN,1500 James Ave N
91,Minneapolis Park and Recreation Board,Waite Park [Mn012200],1810 34th Ave NE,55418,0,Other - Entertainment/Public Assembly,6212,0,1979,47,58,61,123,127,0,NaN,1810 34th Ave NE
59,Minneapolis Park and Recreation Board,Keewaydin Park [Mn012169],3030 53rd St E,55417,0,Other - Entertainment/Public Assembly,6314,0,1972,79,127,137,213,224,0,NaN,3030 53rd St E
96,Minneapolis Public Schools,Andersen Elementary,2727 10th Ave S,55407,41,K-12 School,233557,0,1976,538,79,83,156,161,3381,NaN,2727 10th Ave S


In [ ]:
public_property_info = run_scraper(public_df)

In [37]:
private_df.head()

,Address,Postal Code,ENERGY STAR Score,Primary Property Type,Floor Area (Building) (ft²),Floor Area (Parking) (ft²),Year Built,Total GHG Emissions (Metric Tons CO2e),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Water Use (kgal),Unnamed: 13
0,201 Chicago Ave S,55415,36,Office,62400,0,2012,971,115,118,251,252,708,NaN
1,1011 Nicollet Mall,55403,68,Office,59479,0,1920,986,90,93,256,266,882,NaN
2,4701 Beard Ave S,55410,0,Other,59172,0,1914,1622,157,160,427,428,163,NaN
3,8 60Th St W,55419,76,Worship Facility,60640,62000,1959,268,50,54,77,81,674,NaN
4,700 10Th Ave S,55415,89,Office,60528,0,1920,504,74,77,136,140,414,NaN
